In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# try making own CNN model ✓
# try making output layer with one node ✓
# transforms.HorizontalFlip ✓
# try momentum ✓
# try other optimizers (Adam, Adagrad, Adadelta) ✓
# try other activation fucntions (tanh) ✓

### {"cat":0, "dog":1}

In [ ]:
import zipfile

z= zipfile.ZipFile('../input/dogs-vs-cats/train.zip')
z.extractall()

z1= zipfile.ZipFile('../input/dogs-vs-cats/test1.zip')
z1.extractall()

In [ ]:
from tqdm import tqdm

In [ ]:
train_path = "./train"
test_path = "./test1"

In [ ]:
import os

filepath_train = os.listdir(train_path)
filepath_test1 = os.listdir(test_path)

In [ ]:
filepath_train[1]

In [ ]:
cat=0
dog=0
others=0
total_len=0
for i in tqdm(filepath_train):
    if i.split(".")[0]== "cat":
        cat+=1
    elif i.split(".")[0] == "dog":
        dog+=1
    else:
        others+=1

print(f"Cats: {cat}, Dogs: {dog}, Others: {others}, Length: {cat+dog+others}")

In [ ]:
import cv2

In [ ]:
import matplotlib.pyplot as plt

img_shapes = []

for i in tqdm(range(len(filepath_train))):
    img = cv2.imread(os.path.join(train_path,filepath_train[i]))
    #plt.imshow(img, cmap="gray")
    #plt.show()
    img_shapes.append(img.shape)

In [ ]:
import numpy as np

In [ ]:
len(img_shapes)

In [ ]:
np.argmin(np.array(img_shapes),axis=0)

In [ ]:
img_shapes[np.argmin(np.array(img_shapes),axis=0)[0]]

In [ ]:
img_shapes[np.argmin(np.array(img_shapes),axis=0)[1]]

In [ ]:
np.argmax(np.array(img_shapes),axis=0)

In [ ]:
img_shapes[np.argmax(np.array(img_shapes),axis=0)[0]]

In [ ]:
img_shapes[np.argmax(np.array(img_shapes),axis=0)[1]]

new_reshape = 50#img_shapes[23863][1]
print(new_reshape)

#?cv2.resize

src_img = cv2.imread(os.path.join(train_path,filepath_train[2032]))

src_img.shape

resized_img = cv2.resize(src_img,(new_reshape,new_reshape))

plt.imshow(resized_img,cmap='gray')


plt.imshow(src_img)

In [ ]:
#os.mkdir("./training")

In [ ]:
#os.mkdir("./training/dogs")

In [ ]:
test1 = cv2.imread(os.path.join(test_path,filepath_test1[1]))

In [ ]:
#test1

In [ ]:
type(test1)

In [ ]:
print(filepath_test1[1])
plt.imshow(test1)

In [ ]:
train_slice = int(0.8*len(filepath_train))

In [ ]:
train_slice

In [ ]:
train_split = filepath_train[:train_slice]
val_split = filepath_train[train_slice:]

In [ ]:
len(val_split)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision import transforms
import torch.nn.functional as F

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class DogsVsCatsDataset(Dataset):
    def __init__(self, root, img_list, transform):
        self.root = root
        self.img_list = img_list
        self.transform = transform
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self,index):
        img = cv2.imread(os.path.join(self.root,self.img_list[index]))
        if self.transform:
            img = self.transform(img)
        #img = torch.from_numpy(img)
        label = 0 if self.img_list[index].split(".")[0] == "cat" else 1
        #label = torch.tensor(label)
        
        return img,label
        

In [ ]:
train_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((48,48)), transforms.RandomHorizontalFlip(0.5)])
val_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((48,48))])

In [ ]:
train_data = DogsVsCatsDataset(root = train_path, img_list = train_split, transform=train_transform)

In [ ]:
idx = 420
img,label = train_data[idx]

In [ ]:
print(label)
plt.imshow(img[0])

In [ ]:
val_data = DogsVsCatsDataset(root = train_path, img_list = val_split, transform=val_transform)

In [ ]:
idx = 20
img,label = val_data[idx]

In [ ]:
print(label)
plt.imshow(img[0])

cat=0
dog=0
for img,label in val_data:
    if label == "cat":
        cat += 1
    else:
        dog+=1

print(cat)
print(dog)

cat=0
dog=0
for img,label in test_data:
    if label == "cat":
        cat += 1
    else:
        dog+=1

print(cat)
print(dog)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True) 
val_dataloader = DataLoader(val_data, batch_size=32, shuffle=True) 

In [ ]:
#?torch.nn.Conv2d

In [ ]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3,out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(in_channels=8,out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(in_channels=16,out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pooling = torch.nn.MaxPool2d(2)
        self.linear1 = torch.nn.Linear(6*6*32, 124)
        self.linear2 = torch.nn.Linear(124,32)
        self.linear3 = torch.nn.Linear(32,2)
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        nn.init.xavier_normal_(self.linear3.weight)
        
        
    def forward(self,x):
        x = self.pooling(F.tanh(self.conv1(x)))
        x = self.pooling(F.tanh(self.conv2(x)))
        x = self.pooling(F.tanh(self.conv3(x)))
        x = x.view(-1,6*6*32)
        x = F.tanh(self.linear1(x))
        x = F.tanh(self.linear2(x))
        x = self.linear3(x)
        return x

In [ ]:
model = ConvNet()
model = model.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
learning_rate = 3e-4
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
"""SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0003
    momentum: 0
    nesterov: False
    weight_decay: 0
)"""

In [ ]:
num_epochs = 7

In [ ]:
LOSS = []
ACC = []
correct=0
num_samples = 0
for epoch in tqdm(range(num_epochs)):
    for img,label in train_dataloader:
        img = img.to(device)
        label = label.to(device)
        outputs = model(img)
        loss = criterion(outputs,label)
        optimizer.zero_grad()
    
        loss.backward()
        optimizer.step()
    
        res = torch.max(outputs,axis=1).indices
        #print(outputs)
        #print(res)
        #print(label)
        
    
        correct += (res == label).float().sum()
        num_samples += len(label)
        #print(correct)
        
    acc = correct / num_samples
    LOSS.append(loss.item())
    ACC.append(acc.item())
    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item():.5f}, Accuracy: {acc.item():.2f}")
    #print(loss.item())
    #print(acc.item())
    
print("---------------------------<|[RESULTS]|>---------------------------")    
print(f"LOSS: {LOSS}")
print(f"ACC: {ACC}")
print("--------------------------<|[FINISHED]|>--------------------------")
    
    

In [ ]:
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(LOSS, color=color)
ax1.set_xlabel('epoch', color=color)
ax1.set_ylabel('Cost', color=color)
ax1.tick_params(axis='y', color=color)
    
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('accuracy', color=color) 
ax2.set_xlabel('epoch', color=color)
ax2.plot( ACC, color=color)
ax2.tick_params(axis='y', color=color)
fig.tight_layout()

In [ ]:
with torch.no_grad():
    LOSS1 = []
    ACC1 = []
    correct=0
    num_samples = 0
    for img,label in val_dataloader:
        img = img.to(device)
        label = label.to(device)
        outputs = model(img)
        res = torch.max(outputs,axis=1).indices
        #print(outputs)
        #print(res)
        #print(label)
        correct += (res == label).float().sum()
        num_samples += len(label)
        #print(correct)
        
    acc = correct / num_samples
    LOSS1.append(loss.item())
    ACC1.append(acc)
        
    print(LOSS1)
    print(ACC1)
print("--------------------------FINISHED--------------------------")

In [ ]:
# Relu AdamW Train -> 71.00, Val -> 76.12
# TanH AdamW Train -> 73.59, Val -> 77.78


# Testing

In [ ]:
#filepath_test1

In [ ]:
class DogsVsCatsDatasetTest(Dataset):
    def __init__(self, root, img_list, transform):
        self.root = root
        self.img_list = img_list
        self.transform = transform
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self,index):
        img = cv2.imread(os.path.join(self.root,self.img_list[index]))
        if self.transform:
            img = self.transform(img)
        #img = torch.from_numpy(img)
        #label = 0 if self.img_list[index].split(".")[0] == "cat" else 1
        #label = torch.tensor(label)
        
        return img

In [ ]:
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((48,48))])

In [ ]:
test_dataset = DogsVsCatsDatasetTest(root=test_path, img_list=filepath_test1, transform=test_transform)

In [ ]:
OUTPUTS = []
IMGS = []

for i,img in enumerate(test_dataset):
    
    img = img.to(device)
    img = img.reshape(-1,3,48,48)
    output = model(img)
    
    #OUTPUTS.append(output)
    #IMGS.append(img)
    res = torch.argmax(output)
    if res == 0:
        print(f"--------CAT--------")
    else: 
        print(f"--------DOG--------")
    result_img = img.cpu()
    plt.imshow(result_img[0][0],cmap="gray")
    plt.show()
    
    if i >= 10:
        break

# Summary
### Xavier initializations works better
### Learning Rate of 3e-4 works very well
### Increasing spatial dimension from 48x48 to 96x96 didn't work very well
### Adam Optimizer seems to work slightly better than SGD with momentum(0.9)
### Adagrad 61.01 -> 65.00 Accuracy (seems to work similar to Adam)
### GPU -> 01:00 mins per epoch (approx)
### CPU -> 01:48 mins per epoch (approx)